## Outer and Inner Joins 

"Joins" refer to how two DataFrames are combined. 

To demonstrate, we'll start by showing how this works with NumPy arrays, and then move on to how to do the same things with pandas DataFrames. 

### NumPy

For NumPy, the first step is to create three NumPy arrays, each with different dimensions. To do this we use NumPy's `random.rand()` function, which creates an array with a specified shape, filled with random numbers. We *chain* this with the `.round()` method to shorten the numbers, then multiply the array by 100 to convert the numbers from values less than 1, to values in the 0-100 range (this uses NumPy *broadcasting* to apply the multiplication to each element of the array). This is done to make the numbers easy to look at, as well as demonstrating chaining and broadcasting.

In [30]:
a = np.random.rand(2, 4).round(3)*100
print(a)

[[ 1.1 66.  44.9 12.3]
 [23.3 68.9 39.7 24.4]]


In [31]:
b = np.random.rand(2, 3).round(3)*100
print(b)

[[49.1 20.7 57.2]
 [85.9  0.3 60.6]]


In [32]:
c = np.random.rand(3, 4).round(3)*100
print(c)

[[42.7 35.2  6.1 24.8]
 [69.3 35.6 82.1 17.8]
 [71.2 76.2 23.  66.6]]


Note that although the above arrays are each different shapes - (2, 4), (2, 3), and (3, 4) - there's always one dimension of each array that has the same size as one dimension of another array. For example, `a` and `b` both have 2 rows, while `b` and `c` each have one dimension of length 3, but `b` has 3 columns while `c` has 3 rows. This allows us to have lots of fun combining these arrays in different ways.

NumPy has 'convenience functions' for combining arrays horizontally (adding columns beside columns; `np.hstack()`) and vertically (adding rows below rows; `np.vstack()`). NumPy also has a more generic `np.concatenate()` function that allows either horizontal or vertical concatenation (stacking) using the `axis=` argument. 

### Stacking arrays horizontally

This will produce an array with `b` and `a` together, 'beside' each other. Note that the inputs have to be inside a list:

In [33]:
np.hstack([b, a])

array([[49.1, 20.7, 57.2,  1.1, 66. , 44.9, 12.3],
       [85.9,  0.3, 60.6, 23.3, 68.9, 39.7, 24.4]])

To do the same thing with `np.concatenate()`, we include the `axis=1` argument to specify joining on the column axis. 

In [34]:
np.concatenate([b, a], axis=1)

array([[49.1, 20.7, 57.2,  1.1, 66. , 44.9, 12.3],
       [85.9,  0.3, 60.6, 23.3, 68.9, 39.7, 24.4]])

### Stacking arrays vertically

This will produce an array with `c` stacked below `a`:

In [35]:
np.vstack([a, c])

array([[ 1.1, 66. , 44.9, 12.3],
       [23.3, 68.9, 39.7, 24.4],
       [42.7, 35.2,  6.1, 24.8],
       [69.3, 35.6, 82.1, 17.8],
       [71.2, 76.2, 23. , 66.6]])

Again, we can use `np.concatenate()`, but this time we need to specify `axis=0` to indicate we're stacking on rows:

In [36]:
np.concatenate([a, c], axis=0)

array([[ 1.1, 66. , 44.9, 12.3],
       [23.3, 68.9, 39.7, 24.4],
       [42.7, 35.2,  6.1, 24.8],
       [69.3, 35.6, 82.1, 17.8],
       [71.2, 76.2, 23. , 66.6]])

## Combining pandas arrays

NumPy arrays are relatively easy to work with, because they contain only numbers. Real datasets stored in pandas DataFrames present unique challenges, though, because they contain *labelled data*, and often they containg *missing data*. For instance, in a study an individual human participant may provide data on a number of tests, such as a working memory span, reading comprehension, and nonverbal intelligence. Sometimes, for any number of reasons (e.g., time, technical failures, human error), an individual's data on one test may be missing. As another example, in a reaction time (RT) study each participant will complete a large number of trials, resulting in *repeated measures* from the same individual (RT measures on each trial). On some trials, individuals may fail to respond, resulting in missing data for those trials. 

Missing data creates problems when combining data. Depending on the situation, it may be preferable to replace missing values with null values (which appear in Python as `NaN`, for 'not a number'), or it may be preferable to have complete data and leave out data that we don't have for all of the inputs (e.g., drop the data from one test completely, if we don't have data for every participant).

This is where teh terms **inner join** and **outer join** come in. This is a bit of jargon you need to learn, but it's pretty logical. 

An **outer join** involves filling in missing values with `NaN`. In formal logical terms, this is the *union* of the input data sets. This is the default for `pd.concat()`. You can also think of the names as reflecting the fact that this approach includes all the data within the 'outer' boundaries of the DataFrame, like a box drawn around the entire table.

An **inner join** involves keeping only the data that is complete for all inputs. In formal logic, this is the *intersection* of the inputs (i.e., only what they all have in common). You can think of the term 'inner' as referring to the fact that this takes only the data inside that big outer box, that has no missing data. 

## Example Data

Here we have data from two studies of reading and related abilities in children. Each study involved different children. In each study, some of the same measures were collected (such as vocabulary), along with some that were collected in only one study. We'd like to combine the data from the two studies. 

First, let's load the data from each study and see what we have. Note that I already know that there's a `Participant` column that uniquely identifies each person by an ID code, so we use that as the index for the DataFrame.

In [37]:
study1 = pd.read_csv('study1.csv', index_col='Participant')
study1.shape

(36, 6)

So study 1 contains 6 measures from each of 36 participants. Let's look at how the data are structured:

In [38]:
study1

,Fluency,WordID,Comprehension,Orthoknow,Vocab,PhonAwar
Participant,,,,,,
study1_01,73.0,84.0,41,47.0,32.0,31.0
study1_02,104.0,45.0,34,37.0,32.0,15.0
study1_03,109.0,59.0,20,48.0,31.0,26.0
study1_04,94.0,60.0,38,48.0,33.0,29.0
study1_05,106.0,66.0,41,NaN,34.0,32.0
study1_06,133.0,52.0,48,28.0,41.0,13.0
study1_07,118.0,67.0,39,46.0,39.0,28.0
study1_08,106.0,71.0,25,45.0,37.0,30.0
study1_09,128.0,69.0,35,50.0,29.0,31.0


You can see that, since we read in the data without an `index=` argument, the index defaults to numbers. We might want to use the participant ID as the index, but we'll decide on that later. 

Now let's load the data from the other study and look at it:

In [39]:
study2 = pd.read_csv('study2.csv', index_col='Participant')
study2.shape

(43, 4)

So now we have 4 measures from 43 participants. Again we examine it:

In [40]:
study2

,Comprehension,Vocab,Nonverbal,Fluency
Participant,,,,
study2_01,17.0,15.0,15.0,137.0
study2_02,NaN,31.0,21.0,115.0
study2_05,NaN,NaN,20.0,95.0
study2_07,34.0,7.0,12.0,NaN
study2_08,28.0,18.0,10.0,52.0
study2_09,21.0,27.0,NaN,130.0
study2_10,28.0,NaN,8.0,136.0
study2_11,NaN,26.0,17.0,60.0
study2_12,NaN,26.0,NaN,97.0


Comparing the two datasets, we can see that there are three measures in common across the two studies: `Fluency`, `Comprehension`, and `Vocab`. Each study also has unique measures, for which we don't have data in the other study: study 1 has `WordID`, `Orthoknow`, and `PhonAwar`, while study 2 has `Nonverbal`. 

The other thing to note is that in both datasets, there are missing data (`NaN`); for some participants we are missing data on some tests. 

### Combining the data sets

Now that we have an idea of what we're working with, we can think about how to combine these two datasets using `pd.concat()`. The first question is whether horizontal or vertical concatenation makes more sense. Since each row of data in each data set corresponds to one individual, it really doesn't make sense to combine these horizontally. So we want to concatenate vertically, stacking the rows. For this we use the `axis=0` argument.

In [41]:
studies_1_2 = pd.concat([study1, study2], axis=0)
studies_1_2

,Fluency,WordID,Comprehension,Orthoknow,Vocab,PhonAwar,Nonverbal
Participant,,,,,,,
study1_01,73.0,84.0,41.0,47.0,32.0,31.0,NaN
study1_02,104.0,45.0,34.0,37.0,32.0,15.0,NaN
study1_03,109.0,59.0,20.0,48.0,31.0,26.0,NaN
study1_04,94.0,60.0,38.0,48.0,33.0,29.0,NaN
study1_05,106.0,66.0,41.0,NaN,34.0,32.0,NaN
study1_06,133.0,52.0,48.0,28.0,41.0,13.0,NaN
study1_07,118.0,67.0,39.0,46.0,39.0,28.0,NaN
study1_08,106.0,71.0,25.0,45.0,37.0,30.0,NaN
study1_09,128.0,69.0,35.0,50.0,29.0,31.0,NaN


You can see above that pandas preserved all of the columns from both inputs, combining the data for column labels that existed in both data sets, and inserting `NaN` in any column that was only present in one of the data sets, for the participants who did not provide data on that measure.

Again, this is called an **outer join**, and is the default for `pd.concat()`. In some data analysis situations, we might only want to analyze data from *complete cases* — measures for which there is no missing data. To do this, we would perform an **inner join**, which includes only the data from complete cases:

In [42]:
studies_1_2 = pd.concat([study1, study2], axis=0, join='inner')
studies_1_2

,Fluency,Comprehension,Vocab
Participant,,,
study1_01,73.0,41.0,32.0
study1_02,104.0,34.0,32.0
study1_03,109.0,20.0,31.0
study1_04,94.0,38.0,33.0
study1_05,106.0,41.0,34.0
study1_06,133.0,48.0,41.0
study1_07,118.0,39.0,39.0
study1_08,106.0,25.0,37.0
study1_09,128.0,35.0,29.0


Above you can see that only the `Participant`, `Fluency`, `Comprehension`, and `Vocab` columns were kept; the others were discarded. 

Note however that there are still `NaN` values for some participants, for some measures. In other words, our inner join only applied to the columns and not to the rows. If we truly want complete cases, and therefor wish to drop any participant with missing data, we can use the `.dropna()` method:

In [43]:
studies_1_2.dropna(axis=0)

,Fluency,Comprehension,Vocab
Participant,,,
study1_01,73.0,41.0,32.0
study1_02,104.0,34.0,32.0
study1_03,109.0,20.0,31.0
study1_04,94.0,38.0,33.0
study1_05,106.0,41.0,34.0
study1_06,133.0,48.0,41.0
study1_07,118.0,39.0,39.0
study1_08,106.0,25.0,37.0
study1_09,128.0,35.0,29.0


We can combine this with `pd.concat()` through chaining, to achieve the full result in one line of code:

In [44]:
studies_1_2 = pd.concat([study1, study2], axis=0, join='inner').dropna(axis=0)
studies_1_2

,Fluency,Comprehension,Vocab
Participant,,,
study1_01,73.0,41.0,32.0
study1_02,104.0,34.0,32.0
study1_03,109.0,20.0,31.0
study1_04,94.0,38.0,33.0
study1_05,106.0,41.0,34.0
study1_06,133.0,48.0,41.0
study1_07,118.0,39.0,39.0
study1_08,106.0,25.0,37.0
study1_09,128.0,35.0,29.0
